In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from scipy import interpolate
from scipy import signal
import time
import matplotlib.pyplot as plt

In [2]:
def readWISDM(usuario):
    pasta = "C:\\Users\\Usuario\\Desktop\\wisdm-dataset\\wisdm-dataset\\raw\\phone\\"
    fileA = 'data_' + str(usuario) + '_accel_phone.txt'
    fileG = 'data_' + str(usuario) + '_gyro_phone.txt'
    acc = np.loadtxt(pasta+'accel\\'+fileA, delimiter=',', comments=';', unpack=True,
                     dtype={'names': ('Subject-id', 'Activity Label', 'Timestamp', 'x', 'y', 'z'),
                            'formats': ('i4', 'U1', 'u8', 'f8', 'f8', 'f8')})
    gyr = np.loadtxt(pasta+'gyro\\'+fileG, delimiter=',', comments=';', unpack=True,
                     dtype={'names': ('Subject-id', 'Activity Label', 'Timestamp', 'x', 'y', 'z'),
                            'formats': ('i4', 'U1', 'u8', 'f8', 'f8', 'f8')})
    return acc, gyr;

In [3]:
acc, gyr = readWISDM(1649)
labels = np.unique(acc[1])

In [4]:
px.line(acc[3][:5000])

In [5]:
rotulos = np.array(['walking', 'jogging', 'stairs', 'sitting', 'standing'])

In [6]:
acc, gyr = readWISDM(1629)
A = np.array(acc[3:])
atividade = 0
ind1 = acc[1]==labels[atividade]
fA = A[:,ind1]
t1 = acc[2][ind1]/1000000000
t1 = t1-t1[0]
px.line(fA.transpose())

In [ ]:
h = signal.butter(3, 0.3, 'low', fs=20, output='sos')
zi = signal.sosfilt_zi(h) * fA[0,:4].mean()
sf, zo = signal.sosfilt(h, fA[0,:], zi=zi)
sf2 = signal.sosfiltfilt(h, fA[0,:])
#sf = signal.sosfilt(h, fA[0,:])
fig = go.Figure()
fig.add_trace(go.Scatter(y=fA[0,:], mode='lines'))
fig.add_trace(go.Scatter(y=sf, mode='lines'))
fig.add_trace(go.Scatter(y=sf2, mode='lines'))

In [ ]:
h = np.ones(30)/30
sf = signal.convolve(fA[1,:], h, mode='same')
fig = go.Figure()
fig.add_trace(go.Scatter(y=fA[1,:], mode='lines'))
fig.add_trace(go.Scatter(y=sf, mode='lines'))
fig.add_trace(go.Scatter(y=sf2, mode='lines'))

In [ ]:
if np.any(np.diff(t1)<0):
    pos = np.nonzero(np.diff(t1)<0)[0].astype(int)
    for k in pos:
        t1[k+1:] = t1[k+1:]+t1[k]+1/20

In [ ]:
pos = np.nonzero(np.diff(t1)<0)[0].astype(int)
aux = np.copy(t1)
aux[pos[0]+1:] = aux[pos[0]+1:]+aux[pos[0]]+1/20
px.scatter(aux)

In [7]:
# Standard activity code
conversao = [2, 5, 6, 0, 1]
# Rótulos WISDM
# 0: walking 
# 1: jogging 
# 2: stairs
# 3: sitting
# 4: standing
# Rotulos padronizados
# 0: Sit
# 1: Stand
# 2: Walk 
# 3: Upstairs
# 4: Downstairs
# 5: Run/Jogging
# 6: Stairs

In [45]:
#chunk = []
chunk = np.zeros((54747, 363))
cont = 0

#h = signal.butter(3, 0.3, 'hp', fs=20, output='sos')

mfi1 = []
vfi1 = []
mfi2 = []
vfi2 = []
for usuario in tqdm(range(1600,1651)): #1600
    acc, gyr = readWISDM(usuario)
    A = np.array(acc[3:])
    G = np.array(gyr[3:])
    for atividade in range(5): #18):
        ind1 = acc[1]==labels[atividade]
        ind2 = gyr[1]==labels[atividade]
        if ~np.all(~ind1) and ~np.all(~ind2):
            t1 = acc[2][ind1]/1000000000
            t2 = gyr[2][ind2]/1000000000
            t1 = t1-t1[0]
            t2 = t2-t2[0]
            if np.any(np.diff(t1)<0):
                pos = np.nonzero(np.diff(t1)<0)[0].astype(int)
                for k in pos:
                    t1[k+1:] = t1[k+1:]+t1[k]+1/20
            if np.any(np.diff(t2)<0):
                pos = np.nonzero(np.diff(t2)<0)[0].astype(int)
                for k in pos:
                    t2[k+1:] = t2[k+1:]+t2[k]+1/20
            fA = A[:,ind1]
            fG = G[:,ind2]
            # intp1 = interpolate.interp1d(t1, fA, kind='cubic')
            # intp2 = interpolate.interp1d(t2, fG, kind='cubic')
            # nt1 = np.arange(0,t1[-1],1/20)
            # nt2 = np.arange(0,t2[-1],1/20)
            # fA = intp1(nt1)
            # fG = intp2(nt2)
            
            # fA = signal.sosfiltfilt(h, fA, axis=1)

            #mfi1.append(np.mean(1/np.diff(t1)))
            #vfi1.append(np.std(1/np.diff(t1)))
            #mfi2.append(np.mean(1/np.diff(t2)))
            #vfi2.append(np.std(1/np.diff(t2)))

            NJ = min(t1.shape[0]//60, t2.shape[0]//60)
            aux = np.zeros(60*6+3)
            for i in range(NJ):
                aux[:180] = fA[:,i*60:(i+1)*60].reshape(60*3)
                aux[180:360] = fG[:,i*60:(i+1)*60].reshape(60*3)
                if atividade < 5:
                    aux[-1] = conversao[atividade]
                aux[-2] = usuario
                aux[-3] = atividade
                #chunk.append(aux)
                chunk[cont,:] = aux
                cont += 1

  0%|          | 0/51 [00:00<?, ?it/s]

In [23]:
semgravidade = chunk[:15196,:]

In [24]:
comgravidade = chunk[:15196,:]

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=semgravidade[1310,:360], mode='lines'))
fig.add_trace(go.Scatter(y=comgravidade[1310,:360], mode='lines'))

In [29]:
px.histogram(semgravidade[:,-3])

In [30]:
fig = px.scatter(y=mfi1, error_y=vfi1)
fig.add_trace(go.Scatter(y=mfi2, 
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=vfi2,
            visible=True), mode='markers', showlegend=False))
fig.show()

In [32]:
usuario = np.random.randint(1600,1651)

atividade = np.random.randint(18)
print(usuario,atividade)
acc, gyr = readWISDM(usuario)
t1 = acc[2][acc[1]==labels[atividade]]/1000000000
t2 = gyr[2][gyr[1]==labels[atividade]]/1000000000
t1 = t1-t1[0]
t2 = t2-t2[0]
print(np.mean(1/np.diff(t1)))
print(np.std(1/np.diff(t1)))
fig = go.Figure(layout=go.Layout(height=350, width=900))
fig.add_trace(go.Scatter(x=t1,y=1/np.diff(t1), mode='lines+markers', showlegend=False))
fig.add_trace(go.Scatter(x=t2,y=1/np.diff(t2), mode='lines+markers', showlegend=False))
fig.update_layout(
    margin=dict(l=20, r=20, t=5, b=20)
)
fig.show()

1615 8
19.859393643696613
0.0003845826119763338


In [33]:
colunas = ['accel-x-' + str(x) for x in np.arange(60).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(60).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(60).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(60).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(60).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(60).tolist()]
colunas += ['activity code']
colunas += ['user']
colunas += ['standard activity code']

In [34]:
dfcg = pd.DataFrame(data=comgravidade)
dfcg.columns = colunas
dfcg = dfcg.astype({"activity code": int, "user": int, "standard activity code": int})

In [ ]:
dfsg = pd.DataFrame(data=semgravidade)
dfsg.columns = colunas
dfsg = dfsg.astype({"activity code": int, "user": int, "standard activity code": int})

In [44]:
treinocg = dfcg[dfcg['user']<=1635]
testecg = dfcg[dfcg['user']>1635]
treinocg.shape[0]/(treinocg.shape[0]+testecg.shape[0])
#testecg.shape[0]/(treinocg.shape[0]+testecg.shape[0])

0.7600026322716504

In [ ]:
treinosg = dfsg[dfsg['user']<=1635]
testesg = dfsg[np.logical_and(dfsg['user']>1635, dfsg['user']<=1645)]
validsg = dfsg[dfsg['user']>1645]

In [38]:
treinosg.shape[0]/(treinosg.shape[0]+testesg.shape[0])

NameError: name 'treinosg' is not defined

In [ ]:
folder = 'G:\\Meu Drive\\Doutorado Unicamp\\Projeto\\organizando datasets\\WISDM organizado\\'
sub = 'WISDM_balanced_20Hz_without_gravity\\'
treinosg.to_csv(folder+sub+'train.csv',index=False)
testesg.to_csv(folder+sub+'test.csv',index=False)
validsg.to_csv(folder+sub+'validation.csv',index=False)

In [41]:
folder = 'C:\\Users\\Usuario\\Desktop\\data\\WISDM\\original_view\\'
sub = 'WISDM_balanced_20Hz_with_gravity\\'
treinocg.to_csv(folder+'train.csv',index=False)
testecg.to_csv(folder+'test.csv',index=False)

In [42]:
treinocg

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user,standard activity code
0,-0.364761,-0.879730,2.001495,0.450623,-2.164352,-4.332779,-0.319443,1.566452,-0.323746,-1.811676,...,0.068039,-0.137405,-0.467010,-0.187714,-0.052719,0.050415,0.570847,0,1600,2
1,-0.019897,0.058136,0.805832,0.392441,-3.992523,-4.075867,-0.342834,0.767914,-0.467209,-2.237320,...,0.424530,-0.365845,-0.224045,0.009583,0.365555,0.564865,0.816101,0,1600,2
2,-0.651840,-1.380219,0.191849,-1.169861,-5.252274,0.886673,1.784363,-0.943817,-2.776306,-1.056335,...,-0.100052,-0.302734,-0.605026,0.043823,0.137939,0.786972,0.793564,0,1600,2
3,-2.382492,0.510223,-0.461853,-3.781677,-5.367844,-0.223450,1.547012,-0.414993,-3.513077,-1.619766,...,-0.172592,-0.429535,-0.141785,-0.012558,0.134155,0.945755,0.933792,0,1600,2
4,0.072647,2.348526,-2.201248,-7.240723,-4.361862,-1.856140,0.024414,0.723221,-1.243652,-0.348114,...,-0.236725,-0.413589,-0.586227,-0.115829,0.335556,0.492676,0.721069,0,1600,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11544,-1.099838,-1.528565,-1.469269,-1.407852,-1.413147,-1.324600,-1.148178,-1.006699,-0.880966,-0.731750,...,0.002640,-0.007202,0.000107,-0.003845,-0.010101,-0.003571,-0.004013,4,1635,1
11545,1.183639,1.593979,1.093430,0.102875,-0.028641,0.054779,-0.219604,-0.723770,-0.986587,-1.041046,...,-0.000305,0.001633,-0.004410,0.002136,-0.004639,-0.002853,-0.005386,4,1635,1
11546,-1.829651,-1.624466,-0.657776,0.118988,0.269760,0.136459,-0.173752,-0.631531,-0.705841,-0.546783,...,0.004044,-0.009705,-0.000290,-0.000763,-0.005020,-0.024429,-0.022644,4,1635,1
11547,-0.112076,-0.191681,-0.179901,-0.251846,-0.047394,0.020203,-0.068359,-0.373138,-0.186844,-0.004974,...,0.062897,0.042221,0.031525,0.041870,0.020935,0.012695,0.023117,4,1635,1


In [ ]:
folder = 'C:\\Users\\Narayana\\Desktop\\organizando datasets\\WISDM organizado\\'
df0 = pd.read_csv(folder+'WISDM_V0.csv')
treino = pd.read_csv(folder+'WISDM_V2_train.csv')
teste = pd.read_csv(folder+'WISDM_V2_test.csv')

In [ ]:
print('Fração de treino e teste', '\nTreino: ', treino.shape[0]/(treino.shape[0]+teste.shape[0]),
      '\nTeste: ', teste.shape[0]/(treino.shape[0]+teste.shape[0]))

In [ ]:
treino.to_csv('WISDM_V2_train.csv',index=False)
teste.to_csv('WISDM_V2_test.csv',index=False)

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import umap

In [ ]:
treino = df[df['user']<=1635]
teste = df[df['user']>1635]
treino = treino[treino['activity']<5]
teste = teste[teste['activity']<5]

In [ ]:
print('Fração de treino e teste', '\nTreino: ', treino.shape[0]/(treino.shape[0]+teste.shape[0]),
      '\nTeste: ', teste.shape[0]/(treino.shape[0]+teste.shape[0]))

In [ ]:
treino.to_csv('WISDM_V2_train.csv',index=False)
teste.to_csv('WISDM_V2_test.csv',index=False)

In [ ]:
np.sum(np.absolute(np.fft.fft(x))[:,:30],axis=1)

In [ ]:
E = np.sum(X1[i,:]**2)
print(E,np.sum((X1[i,:]/np.sqrt(E))**2))
px.line(X1[i,:])

In [ ]:
X1 = np.array(treino)[:,:360]
N1 = X1.shape[0]  # Número de amostras
T = 60         # Número de amostras da série temporal
d = 180         # Dimensão do espectro total (acc+gyr)
ns = 6          # Número de séries temporais
SPC1 = np.zeros((N1, d))
SPC10 = np.zeros((N1, d))
SPC1n = np.zeros((N1, d))
SPC1f = np.zeros((N1, 192))
E1 = np.zeros((N1, ns))
xm1 = np.zeros((N1, ns))
for i in range(N1):
    x = X1[i,:].reshape((ns, T))
    xm1[i,:] = np.mean(x,axis=1)
    x0 = x - xm1[i,:].reshape(6,1) @ np.ones((1,60))
    E1[i,:] = np.sqrt(np.sum(x0**2,axis=1))
    xn = x0 / (E1[i,:].reshape((6,1)) @ np.ones((1,60)))
    Xn = np.absolute(np.fft.fft(xn))[:,:30]
    SPC1[i,:] = np.absolute(np.fft.fft(x))[:,:30].reshape(180)
    SPC10[i,:] = np.absolute(np.fft.fft(x0))[:,:30].reshape(180)
    SPC1n[i,:] = np.absolute(np.fft.fft(xn))[:,:30].reshape(180)
    SPC1f[i,:] = np.concatenate((E1[i,:].reshape(ns,1)/1, xm1[i,:].reshape(ns,1)/1, Xn), axis=1).reshape(192)

In [ ]:
X2 = np.array(teste)[:,:360]
N2 = X2.shape[0]  # Número de amostras
T = 60         # Número de amostras da série temporal
d = 180         # Dimensão do espectro total (acc+gyr)
ns = 6          # Número de séries temporais
SPC2 = np.zeros((N2, d))
SPC20 = np.zeros((N2, d))
SPC2n = np.zeros((N2, d))
SPC2f = np.zeros((N2, 192))
E2 = np.zeros((N2, ns))
xm2 = np.zeros((N2, ns))
for i in range(N2):
    x = X2[i,:].reshape((ns, T))
    xm2[i,:] = np.mean(x,axis=1)
    x0 = x - xm2[i,:].reshape(6,1) @ np.ones((1,60))
    E2[i,:] = np.sqrt(np.sum(x0**2,axis=1))
    xn = x0 / (E2[i,:].reshape((6,1)) @ np.ones((1,60)))
    Xn = np.absolute(np.fft.fft(xn))[:,:30]
    SPC2[i,:] = np.absolute(np.fft.fft(x))[:,:30].reshape(180)
    SPC20[i,:] = np.absolute(np.fft.fft(x0))[:,:30].reshape(180)
    SPC2n[i,:] = np.absolute(np.fft.fft(xn))[:,:30].reshape(180)
    SPC2f[i,:] = np.concatenate((E2[i,:].reshape(ns,1)/1, xm2[i,:].reshape(ns,1)/1, Xn), axis=1).reshape(192)

In [ ]:
px.line(SPC2f[:2,:].transpose())

In [ ]:
# Experimentos de classificação
y1 = np.array(treino['activity'])
y2 = np.array(teste['activity'])
clf = KNeighborsClassifier(n_neighbors=15)
#clf = svm.SVC()
#clf = RandomForestClassifier(random_state=0)
clf.fit(SPC1, y1)
yp = clf.predict(SPC2)
f1 = f1_score(y2, yp, average='micro')
acc = accuracy_score(y2, yp)
print('f1: ', f1, '\nAcurácia: ', acc)

In [ ]:
class_names = labels

titles_options = [("Normalized confusion matrix", "true")]

plt.rcParams["figure.figsize"] = (15,15)
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        clf,
        SPC2,
        y2,
        display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize=normalize,xticks_rotation='vertical'
    )
    disp.ax_.set_title(title)
#plt.savefig('rdftempo.jpg', transparent=False, bbox_inches='tight')
plt.show()
#print("\n"+"*"*100)

In [ ]:
reducer1 = umap.UMAP(n_neighbors=10, min_dist=0.9, init='random', random_state=42)
embedding1 = reducer1.fit_transform(SPC1)
reducer10 = umap.UMAP(n_neighbors=10, min_dist=0.9, init='random', random_state=42)
embedding10 = reducer10.fit_transform(SPC10)
reducer1n = umap.UMAP(n_neighbors=10, min_dist=0.9, init='random', random_state=42)
embedding1n = reducer1n.fit_transform(SPC1n)
reducer1f = umap.UMAP(n_neighbors=10, min_dist=0.9, init='random', random_state=42)
embedding1f = reducer1f.fit_transform(SPC1f)

In [ ]:
embedding2 = reducer1.transform(SPC2)
embedding20 = reducer10.transform(SPC20)
embedding2n = reducer1n.transform(SPC2n)
embedding2f = reducer1f.transform(SPC2f)

In [ ]:
ex = np.concatenate((embedding1[:,0], embedding2[:,0]))
ey = np.concatenate((embedding1[:,1], embedding2[:,1]))
labels2 = np.concatenate((labels, np.char.add(labels,'2')))
cor = np.concatenate((y1, y2+5))
px.scatter(x=ex,y=ey, color=labels2[cor], width = 800, height = 500)

In [ ]:
ex = np.concatenate((embedding10[:,0], embedding20[:,0]))
ey = np.concatenate((embedding10[:,1], embedding20[:,1]))
labels2 = np.concatenate((labels, np.char.add(labels,'2')))
cor = np.concatenate((y1, y2+5))
px.scatter(x=ex,y=ey, color=labels2[cor], width = 800, height = 500)

In [ ]:
ex = np.concatenate((embedding1n[:,0], embedding2n[:,0]))
ey = np.concatenate((embedding1n[:,1], embedding2n[:,1]))
labels2 = np.concatenate((labels, np.char.add(labels,'2')))
cor = np.concatenate((y1, y2+5))
px.scatter(x=ex,y=ey, color=labels2[cor], width = 800, height = 500)

In [ ]:
ex = np.concatenate((embedding1f[:,0], embedding2f[:,0]))
ey = np.concatenate((embedding1f[:,1], embedding2f[:,1]))
labels2 = np.concatenate((labels, np.char.add(labels,'2')))
cor = np.concatenate((y1, y2+5))
px.scatter(x=ex,y=ey, color=labels2[cor], width = 800, height = 500)

In [ ]:
# Experimentos de classificação
y1 = np.array(treino['activity'])
y2 = np.array(teste['activity'])
clf = KNeighborsClassifier(n_neighbors=15)
#clf = svm.SVC()
#clf = RandomForestClassifier(random_state=0)
clf.fit(embedding1, y1)
yp = clf.predict(embedding2)
f1 = f1_score(y2, yp, average='micro')
acc = accuracy_score(y2, yp)
print('f1: ', f1, '\nAcurácia: ', acc)

class_names = labels

titles_options = [("Normalized confusion matrix", "true")]

plt.rcParams["figure.figsize"] = (15,15)
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        clf,
        embedding2,
        y2,
        display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize=normalize,xticks_rotation='vertical'
    )
    disp.ax_.set_title(title)
#plt.savefig('rdftempo.jpg', transparent=False, bbox_inches='tight')
plt.show()
#print("\n"+"*"*100)

In [ ]:
# Experimentos de classificação
y1 = np.array(treino['activity'])
y2 = np.array(teste['activity'])
clf = KNeighborsClassifier(n_neighbors=15)
#clf = svm.SVC()
#clf = RandomForestClassifier(random_state=0)
clf.fit(embedding10, y1)
yp = clf.predict(embedding20)
f1 = f1_score(y2, yp, average='micro')
acc = accuracy_score(y2, yp)
print('f1: ', f1, '\nAcurácia: ', acc)

class_names = labels

titles_options = [("Normalized confusion matrix", "true")]

plt.rcParams["figure.figsize"] = (15,15)
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        clf,
        embedding20,
        y2,
        display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize=normalize,xticks_rotation='vertical'
    )
    disp.ax_.set_title(title)
#plt.savefig('rdftempo.jpg', transparent=False, bbox_inches='tight')
plt.show()
#print("\n"+"*"*100)

In [ ]:
# Experimentos de classificação
y1 = np.array(treino['activity'])
y2 = np.array(teste['activity'])
clf = KNeighborsClassifier(n_neighbors=15)
#clf = svm.SVC()
#clf = RandomForestClassifier(random_state=0)
clf.fit(embedding1n, y1)
yp = clf.predict(embedding2n)
f1 = f1_score(y2, yp, average='micro')
acc = accuracy_score(y2, yp)
print('f1: ', f1, '\nAcurácia: ', acc)

class_names = labels

titles_options = [("Normalized confusion matrix", "true")]

plt.rcParams["figure.figsize"] = (15,15)
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        clf,
        embedding2n,
        y2,
        display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize=normalize,xticks_rotation='vertical'
    )
    disp.ax_.set_title(title)
#plt.savefig('rdftempo.jpg', transparent=False, bbox_inches='tight')
plt.show()
#print("\n"+"*"*100)

In [ ]:
# Experimentos de classificação
y1 = np.array(treino['activity'])
y2 = np.array(teste['activity'])
clf = KNeighborsClassifier(n_neighbors=15)
#clf = svm.SVC()
#clf = RandomForestClassifier(random_state=0)
clf.fit(embedding1f, y1)
yp = clf.predict(embedding2f)
f1 = f1_score(y2, yp, average='micro')
acc = accuracy_score(y2, yp)
print('f1: ', f1, '\nAcurácia: ', acc)

class_names = labels

titles_options = [("Normalized confusion matrix", "true")]

plt.rcParams["figure.figsize"] = (15,15)
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        clf,
        embedding2f,
        y2,
        display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize=normalize,xticks_rotation='vertical'
    )
    disp.ax_.set_title(title)
#plt.savefig('rdftempo.jpg', transparent=False, bbox_inches='tight')
plt.show()
#print("\n"+"*"*100)

In [ ]:
px.line(X1[0,:])

In [ ]:
print(np.mean(X1[:,:60]))
print(np.mean(X1[1000,:60]))
fig = px.line(np.mean(X1[:,:60], axis=0))
fig.add_trace(go.Scatter(y=X1[1000,:60], mode='lines', showlegend=False))